# Цифровой прорыв 2021 

## Задача: Музейный фонд

https://cups.mail.ru/ru/tasks/1087

Обучение искусственного интеллекта на распознавание вида или категории предмета музейного фонда при внесении нового предмета в каталог. Метрика F1-score.

Безлайн по текстовым данным. 
55 место, F1 =  

In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import lightgbm as lgb
import re

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier

In [70]:
train = pd.read_csv('train_url_only.csv')

In [71]:
train.head()

,guid,description,typology,url
0,8d49dc72-33d6-4dbc-b269-de6c1fc563b4,Паспортная книжка Краморева Г.Д.Выдана Новозыб...,документы,NaN
1,43854438-31d8-4551-b3bb-73d44c02dc95,"Открытка почтовая: ""Воен. Гр. Дор. Гора Казбек"".",прочие,http://goskatalog.ru/muzfo-imaginator/rest/ima...
2,6e138bdb-c1f7-4d57-8b73-52340a96d31b,Брошюра. Положение о добровольных народных дру...,документы,NaN
3,440c521d-bd15-4b0f-a89b-8b0b160edf04,"Карточка на жиры для рабочих и ИТР, работающих...",документы,NaN
4,b1a6f073-a833-4a08-8c11-48b29d361494,"Листовка ""Молодые рабочие и работницы! Знаете ...",документы,NaN


In [72]:
test = pd.read_csv('test.csv')
test.head()

,guid,description,typology
0,a74014d7-3054-45bd-b284-ac117ccd2001,NaN,NaN
1,783eab83-a48c-4950-996a-386fe93a3ccd,Образец волос. Удмурты,NaN
2,8e7e2514-fab3-4287-be06-aae93c5397e0,NaN,NaN
3,dc1964ce-9248-4fe7-9d4b-c110ac270f64,"фрески фрагмент, фон. Из комплекта: Фресок фра...",NaN
4,1dd23c5e-bf5d-45b7-9894-07bfdf91ae80,"Афиша. ""Королевские цветы"" (муз. А. Рыбникова)...",NaN


In [73]:
print(train.shape[0])
print(test.shape[0])

197202
1223


In [74]:
print(train.description.isna().sum())
print(test.description.isna().sum())

0
547


In [75]:
train.typology.isna().sum()

6

In [76]:
train.dropna(subset = ['typology'], inplace = True)

In [77]:
train.typology.value_counts()

документы                                             30473
фотографии и негативы                                 29661
предметы археологии                                   24062
предметы прикладного искусства, быта и этнографии     23127
предметы нумизматики                                  22994
предметы печатной продукции                           20598
графика                                               15232
прочие                                                10376
редкие книги                                           8253
предметы естественнонаучной коллекции                  4472
живопись                                               3563
предметы техники                                       1526
предметы минералогической коллекции                    1421
оружие                                                  811
скульптура                                              547
предметы этнографии                                      39
документы, редкие книги                 

In [78]:
train['description'] = train['description'].apply(lambda x: x.lower())
train['description'] = train['description'].apply(lambda x: re.sub("[^а-яА-Я]", " ", x))

In [79]:
test.description = test.description.fillna(' ')

In [80]:
test['description'] = test['description'].apply(lambda x: x.lower())
test['description'] = test['description'].apply(lambda x: re.sub("[^а-яА-Я]", " ", x))

In [81]:
all_text = list(train.description) + list(test.description)

In [84]:
vec = TfidfVectorizer()

In [85]:
vec.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [86]:
X = vec.transform(train.description)
y = train.typology

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
models = [LogisticRegression(random_state=1), KNeighborsClassifier(), RidgeClassifier(random_state=1), 
          SGDClassifier(random_state=1), lgb.LGBMClassifier(random_state=42, silent=True)]
for model in models:
    model.fit(X_train, y_train)
    print(f1_score(y_test, model.predict(X_test), average='macro'))

/home/darya/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5270574942112162
0.39192808580707433
0.5792938317026431
0.5436193737178104
0.39246204788145483


In [90]:
test_text = vec.transform(test.description)
model = RidgeClassifier(random_state=1)
model.fit(X, y)


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=1, solver='auto',
                tol=0.001)

In [91]:
pred = model.predict(test_text)

In [92]:
sub = pd.DataFrame()
sub['guid'] = test.iloc[:,0]
sub['typology'] = pred
sub.to_csv('text_baseline.csv', index=False)